In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
from matplotlib import pyplot
import descartes
import seaborn as sns
import datetime as dt
import geoplot as geoplot
from sklearn.preprocessing import MultiLabelBinarizer
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [2]:
airbnb = pd.read_csv('../data/preprocessing/airbnb.csv')

In [3]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2222 entries, 0 to 2221
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   the_geom                        2222 non-null   object 
 1   OBJECTID                        2222 non-null   int64  
 2   geoid                           2222 non-null   int64  
 3   Median Sales Price              2222 non-null   float64
 4   PercentShortTermRentals         2222 non-null   float64
 5   geometry                        2222 non-null   object 
 6   index_right                     2222 non-null   int64  
 7   id                              2222 non-null   int64  
 8   name                            2222 non-null   object 
 9   description                     2222 non-null   object 
 10  neighborhood_overview           2222 non-null   object 
 11  host_name                       2222 non-null   object 
 12  host_location                   22

In [4]:
targets= ['price_person','middle_estimate/month', 'lowest_estimate/month','price']

In [5]:
#this removes features that I won't use for modeling. I'll keep them in a separate dataframe incase I want to use 
#spacial information or NLP in the future. 
reserved_features = ['the_geom', 'geometry','OBJECTID', 'geoid', 'index_right','id', 'latitude', 'longitude', 'name', 'description', 'neighborhood_overview', 'host_name', 'host_location', 'host_about']
reserved_df = airbnb[reserved_features]
airbnb = airbnb.drop(columns = reserved_features)

# Dummy Variables and One Hot Encoding

The first thing I want to do is make a dataframe for linear regression and one for tree based ensembles. For linear regression, the categorical columns will need to be turned into multiple columns for each category. One Hot Encoding does this by turning each category into its own column and Get Dummies does this by making a column for each category - 1. For categories that are mutually exclusive I should use Dummy Variables and for categories that aren't, I'll use one hot encoding.

## One Hot Encoding for Amenities

I'll use multiplabelbinarizer from sklearn preprocessing to make a column for each amenity listed and fill in 1s for airbnbs with that amenity and fill in 0s for aibnbs without.  Since each instance can contain any combination of amenities, this will be like one hot encoding.

In [6]:
airbnb.amenities.head()

0    ["Wifi", "Long term stays allowed", "Hair drye...
1    ["Wifi", "Long term stays allowed", "Hair drye...
2    ["Lockbox", "Wifi", "Long term stays allowed",...
3    ["Wifi", "Long term stays allowed", "Hair drye...
4    ["Lockbox", "Wifi", "Long term stays allowed",...
Name: amenities, dtype: object

In [7]:
#prepping amenities colulumn to work with multilabelbinarizer
#binarizer method works by separating the words in one string into categories that do not repeat. 
#so I have to remove:
#the brakets
airbnb['amenities']=airbnb['amenities'].map(lambda x: x.lstrip('[').rstrip(']'))
#the quotation marks 
airbnb['amenities']=airbnb['amenities'].map(lambda x: x.replace('"',''))
#and leading whitespaces
airbnb['amenities']=airbnb['amenities'].map(lambda x: x.replace(', ',','))

In [8]:
print(airbnb.amenities[:5])

0    Wifi,Long term stays allowed,Hair dryer,Shampo...
1    Wifi,Long term stays allowed,Hair dryer,Shampo...
2    Lockbox,Wifi,Long term stays allowed,Hair drye...
3    Wifi,Long term stays allowed,Hair dryer,Shampo...
4    Lockbox,Wifi,Long term stays allowed,Hair drye...
Name: amenities, dtype: object


In [9]:
#creates a new data frame of amenities as separate columns with boolean values
mlb = MultiLabelBinarizer()
amenities_df = pd.DataFrame(mlb.fit_transform(airbnb['amenities'].str.split(',')),columns=mlb.classes_)
print(amenities_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2222 entries, 0 to 2221
Data columns (total 94 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   Air conditioning                2222 non-null   int64
 1   BBQ grill                       2222 non-null   int64
 2   Baby bath                       2222 non-null   int64
 3   Baby monitor                    2222 non-null   int64
 4   Babysitter recommendations      2222 non-null   int64
 5   Baking sheet                    2222 non-null   int64
 6   Barbecue utensils               2222 non-null   int64
 7   Bathroom essentials             2222 non-null   int64
 8   Bathtub                         2222 non-null   int64
 9   Beach essentials                2222 non-null   int64
 10  Beachfront                      2222 non-null   int64
 11  Bed linens                      2222 non-null   int64
 12  Bedroom comforts                2222 non-null   int64
 13  Bre

In [10]:
#checks to see if any amenities are used by each airbnb and therefore have no variance
col_list=[]
for col in amenities_df.columns:    
    if amenities_df[col].nunique()<2:
        col_list.append(col)
print("There are " +str(len(col_list)) + ' columns with fewer than 2 unique values')

There are 0 columns with fewer than 2 unique values


In [11]:
#joins main dataframe with amenities dataframe with a prefix. 
airbnb = airbnb.reset_index()
airbnb = airbnb.join(amenities_df.add_prefix('amen_'))


## Finding and Transforming Other Categorical Features


In [12]:
#first change boolean columns to integers
boolean = airbnb.select_dtypes(bool).columns
for col in airbnb[boolean]:
    airbnb[col]=np.multiply(airbnb[col], 1)

#then make a dataframe to look at columns that need to be encoded by removing numeric
numeric = airbnb.select_dtypes(np.number).columns
other = airbnb.drop(columns = numeric)
other.dtypes

host_response_time        object
neighbourhood_cleansed    object
property_type             object
amenities                 object
dtype: object

In [13]:
#dropping amenities since it already has been encoded. 
other.drop(columns = ['amenities','property_type'], inplace = True)

#making dummy columns with prefixes
dummy_columns = other.columns
prefixes = ['resp_tim_','nbhd_']
prefixes = dict(zip(dummy_columns, prefixes))
other = pd.get_dummies(data = other, prefix = prefixes, drop_first = True)

In [14]:
linear_data = airbnb[numeric].join(other)
linear_data.drop(columns = 'index', inplace = True)
print(linear_data.shape)
linear_data.head()

(2222, 193)


,Median Sales Price,PercentShortTermRentals,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,accommodates,bathrooms_text,bedrooms,...,nbhd__St. Roch,nbhd__St. Thomas Dev,nbhd__Tall Timbers - Brechtel,nbhd__Touro,nbhd__Treme - Lafitte,nbhd__Tulane - Gravier,nbhd__Uptown,nbhd__West End,nbhd__West Riverside,nbhd__Whitney
0,134250.0,4.626335,100.0,100.0,1,1,1,8,2.0,4.0,...,0,0,0,0,1,0,0,0,0,0
1,134250.0,4.626335,96.0,98.0,0,1,1,6,2.5,3.0,...,0,0,0,0,1,0,0,0,0,0
2,134250.0,4.626335,100.0,84.0,1,1,1,4,1.0,2.0,...,0,0,0,0,1,0,0,0,0,0
3,134250.0,4.626335,95.0,99.0,0,1,1,6,2.5,3.0,...,0,0,0,0,1,0,0,0,0,0
4,134250.0,4.626335,100.0,100.0,0,1,1,8,2.0,3.0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
print(airbnb.shape)
airbnb.head()

(2222, 135)


,index,Median Sales Price,PercentShortTermRentals,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,...,amen_Smoking allowed,amen_Stair gates,amen_Stove,amen_Suitable for events,amen_TV,amen_Table corner guards,amen_Washer,amen_Waterfront,amen_Wifi,amen_Window guards
0,0,134250.0,4.626335,within an hour,100.0,100.0,1,1,1,Treme - Lafitte,...,0,0,1,0,1,0,1,0,1,0
1,1,134250.0,4.626335,within an hour,96.0,98.0,0,1,1,Treme - Lafitte,...,0,0,1,0,1,0,1,0,1,0
2,2,134250.0,4.626335,within an hour,100.0,84.0,1,1,1,Treme - Lafitte,...,0,0,0,0,1,0,0,0,1,0
3,3,134250.0,4.626335,within an hour,95.0,99.0,0,1,1,Treme - Lafitte,...,0,0,1,0,1,0,1,0,1,0
4,4,134250.0,4.626335,within an hour,100.0,100.0,0,1,1,Treme - Lafitte,...,0,0,0,0,1,0,1,0,1,0


## Data Split for Decision Tree or Linear Regression
The linear_data dataset is now completely encoded for linear regression.
The airbnb dataset can be used for decision tree type regressions such as the random forest regressor. The amenities are encoded, but the other categorical and boolean features are kept as they are. 










# Feature Selection

Encoding created a lot of features. The following will try to address feature selection.  I would like to select features rather than extract with PCA.  I've chosen homeowners as my target audience and I want to stick with methods that easily show what aspects of a rental affect its price. 

## Colinearity

Linear regression will be negatively affected by co and multilinearity in independant features. The following section will address this issue. 

In [16]:
corr=linear_data.drop(columns = targets).corr().abs()
corr = corr.unstack().sort_values().reset_index()
corr = corr[corr['level_0']!=corr['level_1']]#removes columns correlated with themselves from analysis

#takes the correlated data and removes duplicates. otherwise data frame will have feature 1 - feature 2 and feature 2 - feature 1
corr['ordered-cols'] = corr.apply(lambda x: '-'.join(sorted([x['level_0'],x['level_1']])),axis=1)
corr = corr.drop_duplicates(['ordered-cols'])
corr.drop(['ordered-cols'], axis=1, inplace=True)

In [17]:
print(corr.shape)
print(corr.tail(5))

(17766, 3)
                     level_0                          level_1         0
35522        availability_30                  availability_60  0.936361
35524       Restaurant_Count  nbhd__Central Business District  0.937354
35526        availability_60                  availability_90  0.973160
35528            amen_Washer                       amen_Dryer  0.973593
35579  amen_Bedroom comforts         amen_Bathroom essentials  1.000000


In [18]:
#finds correlation price_person has with each feature for comparison
price_0 = []
price_1 = []#lists for storing results
for i, r in corr.iterrows():
    column0 = corr.level_0[i]
    column1 = corr.level_1[i]
    price_0.append(abs(linear_data['price_person'].corr(linear_data[column0])))
    price_1.append(abs(linear_data['price_person'].corr(linear_data[column1])))


In [21]:
#makes a new columns to compare price correlation with feature 0 with price correlation with feature 1. 
corr = corr.reset_index()
corr['price_0'] = pd.Series(price_0)
corr['price_1'] = pd.Series(price_1)

In [39]:
corr[corr[0]>.7]#filters rows with high colinearity
remove = []#starts a list for storing values

for i, row in corr[corr[0]>.7].iterrows():#loop that appends the feature with the smaller correlation to price 
    if corr.price_0[i]<corr.price_1[i]:
        remove.append(corr.level_0[i])
    else:
        remove.append(corr.level_1[i])
remove = set(remove)#makes list unique
print(remove)
print(len(remove))

{'number_of_reviews', 'amen_Refrigerator', 'review_scores_accuracy', 'availability_30', 'amen_Essentials', 'resp_tim__within a few hours', 'amen_Cooking basics', 'amen_Self check-in', 'amen_Dishes and silverware', 'amen_Stove', 'availability_60', 'bedrooms', 'Restaurant_Count', 'amen_Elevator', 'amen_Dryer', 'neighborhood_restaurants', 'bathrooms_text', 'amen_Bathroom essentials', 'review_scores_value', 'beds', 'amen_Full kitchen'}
21


In [61]:
lin_reg_data = linear_data.drop(columns = remove)#makes a new data frame that removes colinear features.
price_corr = {}#dictionary for storing
for column in lin_reg_data.drop(columns = targets).columns:#iterates over columns other than targets
    price_corr[column] = abs(lin_reg_data['price_person'].corr(lin_reg_data[column]))#finds correlation between column and price. 

price_corr = pd.DataFrame(price_corr.items())#stores dict to dataframe for easier filtering

In [68]:
price_corr = price_corr.sort_values(1, ascending = False)#sort values by descending correlation to price
highest_20 = list(price_corr[0][:20])#list to keep 20 most correlated values 
lin_reg_data = lin_reg_data[highest_20]

## Conclusion
There should now be appropriate features for decision tree regressions, standard linear regression and lasso regressions. 

In [ ]:
airbnb.to_csv(r'/Users/John/Desktop/Capstone_Two/data/modeling/decision_tree.csv')
lin_reg_data.to_csv()